## Load all the Python Libraries Required

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
import keras.backend as K
from tensorflow.python.framework import graph_util, graph_io
import coremltools

Using TensorFlow backend.


### Set the learning phase using the backend as False and load the model

In [2]:
K.set_learning_phase(False)

# Load the model with the weights trained on Imagenet 
# and input shape as specified for the model
model = InceptionV3(input_shape=(299,299,3), 
                    weights='imagenet',
                    include_top=True)

### Convert the model into a coreml model

In [3]:
# Convert the loaded Keras model into a CoreML Model
# Provide image_input_names to specify it is an image model
# also provide the bias values specified by the model
mlmodel = coremltools.converters.keras.convert(model, 
                                               input_names=['image'], 
                                               image_input_names=['image'],
                                               output_names=['classLabelProbs'],
                                               class_labels='labels.txt', 
                                               blue_bias=-1,
                                               green_bias=-1,
                                               red_bias=-1,
                                               image_scale=2./255)

0 : input_1, <keras.engine.topology.InputLayer object at 0xd20ba7940>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0xd20ba7dd8>
2 : batch_normalization_1, <keras.layers.normalization.BatchNormalization object at 0xd20ba7b00>
3 : activation_1, <keras.layers.core.Activation object at 0xd20ba7d68>
4 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x10413f9e8>
5 : batch_normalization_2, <keras.layers.normalization.BatchNormalization object at 0xd20bcf160>
6 : activation_2, <keras.layers.core.Activation object at 0xd20c763c8>
7 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0xd20cdb9e8>
8 : batch_normalization_3, <keras.layers.normalization.BatchNormalization object at 0xd20cdbac8>
9 : activation_3, <keras.layers.core.Activation object at 0xd20d5a9e8>
10 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0xd20df84e0>
11 : conv2d_4, <keras.layers.convolutional.Conv2D object at 0xd20db9cf8>
12 : batch_normalization_4, <keras.layers.normalization.B

105 : batch_normalization_36, <keras.layers.normalization.BatchNormalization object at 0xd22d97dd8>
106 : activation_36, <keras.layers.core.Activation object at 0xd22eab4a8>
107 : conv2d_32, <keras.layers.convolutional.Conv2D object at 0xd22a63d68>
108 : conv2d_37, <keras.layers.convolutional.Conv2D object at 0xd22f46c50>
109 : batch_normalization_32, <keras.layers.normalization.BatchNormalization object at 0xd22a15710>
110 : batch_normalization_37, <keras.layers.normalization.BatchNormalization object at 0xd22f0add8>
111 : activation_32, <keras.layers.core.Activation object at 0xd22a8de80>
112 : activation_37, <keras.layers.core.Activation object at 0xd22fbc0b8>
113 : conv2d_33, <keras.layers.convolutional.Conv2D object at 0xd22b23978>
114 : conv2d_38, <keras.layers.convolutional.Conv2D object at 0xd23023a20>
115 : batch_normalization_33, <keras.layers.normalization.BatchNormalization object at 0xd22abc438>
116 : batch_normalization_38, <keras.layers.normalization.BatchNormalization o

205 : batch_normalization_62, <keras.layers.normalization.BatchNormalization object at 0xd24906ac8>
206 : batch_normalization_67, <keras.layers.normalization.BatchNormalization object at 0xd24dfb7b8>
207 : activation_62, <keras.layers.core.Activation object at 0xd24946be0>
208 : activation_67, <keras.layers.core.Activation object at 0xd24e7c828>
209 : conv2d_63, <keras.layers.convolutional.Conv2D object at 0xd24977748>
210 : conv2d_68, <keras.layers.convolutional.Conv2D object at 0xd24f1ce80>
211 : batch_normalization_63, <keras.layers.normalization.BatchNormalization object at 0xd24a32da0>
212 : batch_normalization_68, <keras.layers.normalization.BatchNormalization object at 0xd24edda90>
213 : activation_63, <keras.layers.core.Activation object at 0xd24a8e358>
214 : activation_68, <keras.layers.core.Activation object at 0xd24f8f2b0>
215 : average_pooling2d_7, <keras.layers.pooling.AveragePooling2D object at 0xd250d7eb8>
216 : conv2d_61, <keras.layers.convolutional.Conv2D object at 0xd

313 : predictions__activation__, <keras.layers.core.Activation object at 0xb206408d0>


### Save the model as a .mlmodel file which can be deployed

In [4]:
mlmodel.save('ModelZoo/IncpetionV3.mlmodel')

### Now extract the model using the TensorFlow session to convert it for Android

In [5]:
S = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(S, 
                                                           S.graph.as_graph_def(),
                                                           ['predictions/Softmax'])
graph_io.write_graph(constant_graph, 'ModelZoo/', 'InceptionV3.pb', as_text=False)

INFO:tensorflow:Froze 378 variables.


INFO:tensorflow:Froze 378 variables.


INFO:tensorflow:Converted 378 variables to const ops.


INFO:tensorflow:Converted 378 variables to const ops.


'ModelZoo/InceptionV3.pb'

### To convert the model for TensorFlow Lite

In [6]:
# To convert the model for tensorflow lite, we can save the keras model
# and then convert it into a TensorFlow lite model directly

input_tensor = tf.keras.layers.Input(shape=(299,299,3))
model = tf.keras.applications.inception_v3.InceptionV3(
    weights='imagenet', input_tensor=input_tensor, include_top=True
)

keras_file = 'ModelZoo/InceptionV3.h5'
tf.keras.models.save_model(model, keras_file)

converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()
open('ModelZoo/'+'InceptionV3.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Froze 378 variables.


INFO:tensorflow:Froze 378 variables.


INFO:tensorflow:Converted 378 variables to const ops.


INFO:tensorflow:Converted 378 variables to const ops.


95306148

### Now load the model and get the details of the converted model
### These are useful when deploying the models on Android

In [7]:
interpreter = tf.contrib.lite.Interpreter(model_path='ModelZoo/InceptionV3.tflite')

interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details, output_details)

[{'name': 'input_1', 'index': 295, 'shape': array([  1, 299, 299,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}] [{'name': 'predictions/Softmax', 'index': 315, 'shape': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
